# SHA-1 algorithm

In [5]:
import numpy as np
import binascii

## Help functions

In [3]:
LENGTH_BITS = 64
BLOCK_SIZE = 512

In [10]:
def str_to_bits(string: str) -> np.ndarray:
    bytes_array = bytes(string, 'utf-8')
    array = np.frombuffer(bytes_array, dtype=np.uint8)
    bits_array = np.unpackbits(array)
    return bits_array

def bits_to_str(bits: np.ndarray) -> str:
    bits_array = np.packbits(bits)
    return bits_array.tobytes().decode('utf-8')


def int_to_x_bits(number: int, bits) -> np.ndarray:
    binary_string = np.binary_repr(number, width=bits)
    return np.array(list(binary_string), dtype=np.uint8)

def int_to_64_bits(number: int) -> np.ndarray:
    return int_to_x_bits(number, 64)

def int_to_32_bits(number: int) -> np.ndarray:
    return int_to_x_bits(number, 32)

def bits_to_int(bits_array):
    binary_string = ''.join(bits_array.astype(str))
    return int(binary_string, 2)

def binary_negation(bits: np.ndarray) -> np.ndarray:
    return np.vectorize(lambda x: (x + 1) % 2)(bits)

def add_padding(bits: np.ndarray) -> np.ndarray:
    bits_len = len(bits)
    padding_len = BLOCK_SIZE - (bits_len + LENGTH_BITS + 1) % BLOCK_SIZE
    one_padding = np.ones(1)
    padding = np.zeros(padding_len)
    bits_with_one_padding = np.append(bits, one_padding)
    bits_with_padding = np.append(bits_with_one_padding, padding)
    return bits_with_padding

def add_length_bits(bits: np.ndarray, bits_len: int) -> np.ndarray:
    length_bits = int_to_64_bits(bits_len)
    bits_with_length = np.append(bits, length_bits)
    return bits_with_length

def split_x_blocks(bits: np.ndarray, size: int) -> np.ndarray:
    assert bits.size % size == 0
    num_blocks = bits.size // size
    blocks = np.array_split(bits, num_blocks)
    return np.array(blocks, dtype=np.uint8)

def split_512_blocks(bits: np.ndarray) -> np.ndarray:
    return split_x_blocks(bits, 512)

def split_32_blocks(bits: np.ndarray) -> np.ndarray:
    return split_x_blocks(bits, 32)

def left_rotate(arr, rotations):
    rotations = rotations % arr.size
    return np.roll(arr, -rotations)


In [7]:
h0 = 0x67452301
h1 = 0xEFCDAB89
h2 = 0x98BADCFE
h3 = 0x10325476
h4 = 0xC3D2E1F0

In [ ]:
def extend_chunks(chunks: np.ndarray) -> np.ndarray:
   for i in range(16, 80):
      new_chunk = chunks[i-3] ^ chunks[i-8] ^ chunks[i-14] ^ chunks[i-16]
      new_chunk = left_rotate(new_chunk, 1)
      chunks = np.append(chunks, new_chunk)
   return chunks

def process_block(block, var):
   chunks =  split_32_blocks(block)
   chunks = extend_chunks(chunks)

In [12]:
def sha1(data):
    bits = str_to_bits(data)
    bits_len = len(bits)
    bits = add_padding(bits)
    bits = add_length_bits(bits, bits_len)
    blocks = split_512_blocks(bits)

    for block in blocks:
        var = process_block(block, var)